In [2]:
import pandas  as pd
import matplotlib.pyplot as plt
import datetime
import numpy as np
import plotly.graph_objects as go
from typing import Tuple, Optional
from sklearn.model_selection import train_test_split
import tensorflow as tf
import plotly.graph_objects as go
from pathlib import Path
import common
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from sklearn.preprocessing import MinMaxScaler


Bad key "text.kerning_factor" on line 4 in
/Users/alon/opt/anaconda3/envs/data_science/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution
Using TensorFlow backend.


# Plan:

1. add spy data that matches this data points. 
2. add features to the data. moving avaregase, rate of change. also for the spy data.
3. label the data. ideas: 
    * rate of change in the folowwing X day. (?)
    * max rate of tchange in the neer 5 mins window. this would be helpfull for limit orders - for both long & short. we could place limit order in that rate of change, letting the stock get there "whenever it wants". 
   
  


In [3]:
df = common.get_stock_data('AAPL')
df = common.clean_data(df)
common.add_times(df, 'datetime')
del df['datetime']
del df['minute']
df


,high,low,open,close,average,volume,notional,numberOfTrades,hour,day,month,minute_of_day,day_of_week
0,191.645,190.600,191.645,190.650,191.189,4320.0,825935.940,44.0,9,1,4,570,0
1,190.980,190.640,190.700,190.980,190.761,3246.0,619210.510,32.0,9,1,4,571,0
2,191.090,190.780,191.060,190.930,190.951,2253.0,430211.740,30.0,9,1,4,572,0
3,191.010,190.760,190.980,190.830,190.946,2241.0,427911.290,27.0,9,1,4,573,0
4,190.760,190.600,190.760,190.700,190.666,1069.0,203822.465,12.0,9,1,4,574,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
102585,315.800,315.550,315.605,315.800,315.691,856.0,270231.890,12.0,15,11,5,955,0
102586,315.945,315.670,315.700,315.940,315.852,3833.0,1210661.825,16.0,15,11,5,956,0
102587,315.990,315.870,315.980,315.965,315.925,4685.0,1480107.600,26.0,15,11,5,957,0
102588,315.960,315.735,315.930,315.740,315.816,915.0,288971.565,13.0,15,11,5,958,0


In [ ]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 200
data_examples = []
data_targets = []
for i in range(0, len(df_normelized) - seq_length, 1):
	features = df_normelized.iloc[i:i + seq_length].values
	target = df_normelized['average'][i + seq_length]
	data_examples.append(features)
	data_targets.append(target)
print("Total Patterns: ", len(data_examples))

In [ ]:
# reshape X to be [samples, time steps, features]
examples = numpy.reshape(data_examples, (len(data_examples), seq_length, 13))
targets = np.array(data_targets)
print(examples.shape)
print(targets.shape)

In [ ]:
def plot_single_example(single_example, single_target, prediction=None):
    x_axis_values_up_to_zero = list(range(-len(single_example), 0))
    training_prices = [x[4]
                       for x in single_example]  # the average price is in index 5

    x_axis_values_from_zero = [1]

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x_axis_values_up_to_zero, y=training_prices,
                             mode='lines',
                             name='history'))
    fig.add_trace(go.Scatter(x=x_axis_values_from_zero, y=[single_target],
                             mode='markers',
                             name='real future'))

    if prediction is not None:
        fig.add_trace(go.Scatter(x=x_axis_values_from_zero, y=[prediction],
                                 mode='markers',
                                 name='predicted future'))

    fig.show()


plot_single_example(examples[1000], targets[1000])

In [ ]:
# define the LSTM model
model = Sequential()
model.add(LSTM(500, input_shape=(examples.shape[1], examples.shape[2]), return_sequences=True))
model.add(LSTM(500, return_sequences=True))
model.add(LSTM(500))
# model.add(Dropout(0.2))
model.add(Dense(1, activation='softmax'))
model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01))

In [ ]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
training_indexes = np.random.choice(len(examples), size=100, replace=False)
training_examples, training_targets = examples[training_indexes], targets[training_indexes]

In [ ]:
model.fit(training_examples, 
          training_targets, 
          epochs=20, 
          batch_size=10, 
          callbacks=callbacks_list)